In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.classify.maxent import MaxentClassifier
from sklearn.model_selection import train_test_split
import nltk
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import gc

from catboost import CatBoostClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('../input/simpredict/train.csv')
df['type'] = df['id'].str[:10]
df = df[df['type'] == 'sodepami_v'].sample(629000)
# df1 = df[df['price_vnd']>=6.6e6]
# df = df[df['price_vnd'] < 6.6e6].sample(600000)
# df = pd.concat([df,df1])

In [4]:
df.shape

(629000, 5)

In [5]:
# df1 = pd.read_csv('../input/simpredict/train.csv')
# df1 = df1[df1['price_vnd']>=6.6e6]

In [6]:
# df = pd.concat([df,df1])

In [7]:
df.shape

(629000, 5)

In [8]:
def removing_wrong_phone_number(df):
    df['phone_number'] = df['phone_number'].astype(str)
    list_number = df['phone_number'].to_list()
    list_len = []

    for i in range(len(list_number)):
        list_len.append(len(list_number[i]))

    df['length'] = list_len

    df = df[df['length'] != 11]

    return df

In [9]:
df = removing_wrong_phone_number(df)

In [10]:
def creating_10_feats_from_single_pos(df):
    list_number = df['phone_number'].to_list()
    list_char = np.full((df.shape[0], 9), "-1")

    for i in range(len(list_number)):
        k = 8
        for j in range(len(list_number[i])-1, 0, -1):
            list_char[i][k] = list_number[i][j]
            k-=1
    
            if(k==0):
                list_char[i][k] = list_number[i][0:j]
                break
    for i in range(9):
        df['pos_' + str(i+1)] = list_char[:,i]
    return df

In [11]:
df = creating_10_feats_from_single_pos(df)
df.head(10)

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9
818427,559367,sodepami_vn_0335984554,335984554,699000,sodepami_v,9,3,3,5,9,8,4,5,5,4
196956,750893,sodepami_vn_0325036668,325036668,5000000,sodepami_v,9,3,2,5,0,3,6,6,6,8
631141,998968,sodepami_vn_0987672989,987672989,1850000,sodepami_v,9,9,8,7,6,7,2,9,8,9
304912,511809,sodepami_vn_0785863355,785863355,850000,sodepami_v,9,7,8,5,8,6,3,3,5,5
751212,439052,sodepami_vn_0984313012,984313012,1550000,sodepami_v,9,9,8,4,3,1,3,0,1,2
561075,470974,sodepami_vn_0772861666,772861666,4000000,sodepami_v,9,7,7,2,8,6,1,6,6,6
342615,1027522,sodepami_vn_0522015968,522015968,550000,sodepami_v,9,5,2,2,0,1,5,9,6,8
394965,847436,sodepami_vn_0342868909,342868909,800000,sodepami_v,9,3,4,2,8,6,8,9,0,9
818323,730000,sodepami_vn_0369781981,369781981,3000000,sodepami_v,9,3,6,9,7,8,1,9,8,1
179974,903858,sodepami_vn_0346240066,346240066,499000,sodepami_v,9,3,4,6,2,4,0,0,6,6


In [12]:
def features_from_counting_numbers(df):
    list_number = df['phone_number'].to_list()
    
    list_count = np.full((df.shape[0], 10), 0)
    
    for i in range(len(list_number)):
        for j in range(len(list_number[i])):
            tmp = int(list_number[i][j])
            list_count[i][tmp] += 1  
    for i in range(10):
        df['count_' + str(i)] = list_count[:,i]
    
    max_count = [0 for i in range(len(list_number))]
    for i in range(len(list_number)):
        max_c = 0
        pos = 0
        for j in range(len(list_count[i])):
            if list_count[i][j] >= max_c:
                max_c = list_count[i][j]
                pos = j
        #print(max_c)
        max_count[i] = pos  
        
    df['max_count'] = max_count
    return df

In [13]:
df = features_from_counting_numbers(df)

In [14]:
def combo_feat(df):
    list_number = df['phone_number'].tolist()

    list_2_last = list() #extract 2 last digit

    list_3_last = list() #extract 3 last digit

    list_4_last = list() #extract 4 last digit

    list_5_last = list() #extract 5 last digit

    list_first = list() #extract first digit after 0

    list_combo_3_identical_bool = list() # the phone number contain 111, 222, ....999 or not

    list_combo_3_identical_cat = list() # the phone number contain which one of 111, 222, ....999

    list_combo_4_identical_bool = list() # the phone number contain 1111, 2222, ... or not

    list_combo_4_identical_cat = list()

    list_combo_5_identical_bool = list() # the phone number contain 11111, 22222, ... or not

    list_combo_5_identical_cat = list()

    list_combo_6_identical_bool = list() 

    list_combo_6_identical_cat = list() 

    list_3_combo_s = list() # tam hoa <5

    list_3_combo_57 = list() # tam hoa 5,7

    list_3_combo_689 = list() # tam hoa 6,8,9

    list_4_combo_s = list() # tu quy <5

    list_4_combo_57 = list() # tu quy 5,7

    list_4_combo_689 = list() # tu quy 6,8,9


    for i in range(len(list_number)):
            str_tmp_2_last = list_number[i][-2:]
            list_2_last.append(str_tmp_2_last)

            str_tmp_3_last = list_number[i][-3:]
            list_3_last.append(str_tmp_3_last)

            str_tmp_4_last = list_number[i][-4:]
            list_4_last.append(str_tmp_4_last)

            str_tmp_5_last = list_number[i][-5:]
            list_5_last.append(str_tmp_5_last)

            str_tmp_first = list_number[i][0]
            list_first.append(str_tmp_first)

            list_combo_3 = {'000','111','222','333','444','555','666','777','888','999'}
            combo_3_score = 0
            combo_3_cat = ""

            combo_3_s = 0
            combo_3_57 = 0
            combo_3_689 = 0

            for str_tmp in list_combo_3:
                if str_tmp in list_number[i]:
                    combo_3_score = combo_3_score + 1
                    combo_3_cat += str_tmp

                    x = (int)(str_tmp[0])

                    if(x in {0,1,2,3,4} ): 
                        combo_3_s = combo_3_s + 1
                    elif (x == 5 or x == 7 ):
                        combo_3_57 = combo_3_57 + 1    
                    else:
                        combo_3_689 = combo_3_689 + 1

            list_combo_3_identical_bool.append(combo_3_score)
            list_combo_3_identical_cat.append(combo_3_cat)

            list_3_combo_s.append(combo_3_s)
            list_3_combo_57.append(combo_3_57)
            list_3_combo_689.append(combo_3_689)

            list_combo_4 = {'0000','1111','2222','3333','4444','5555','6666','7777','8888','9999'}
            combo_4_score = 0
            combo_4_cat = ""

            combo_4_s = 0
            combo_4_57 = 0
            combo_4_689 = 0
            for str_tmp in list_combo_4:
                if str_tmp in list_number[i]:
                    combo_4_score = combo_4_score + 1
                    combo_4_cat += str_tmp

                    x = (int)(str_tmp[0])

                    if(x in {0,1,2,3,4} ): 
                        combo_4_s = combo_4_s + 1
                    elif (x == 5 or x == 7 ):
                        combo_4_57 = combo_4_57 + 1    
                    else:
                        combo_4_689 = combo_4_689 + 1            


            list_combo_4_identical_bool.append(combo_4_score)
            list_combo_4_identical_cat.append(combo_4_cat)

            list_4_combo_s.append(combo_4_s)
            list_4_combo_57.append(combo_4_57)
            list_4_combo_689.append(combo_4_689)

            list_combo_5 = {'00000','11111','22222','33333','44444','55555','66666','77777','88888','99999'}
            combo_5_score = 0
            combo_5_cat = ""
            for str_tmp in list_combo_5:
                if str_tmp in list_number[i]:
                    combo_5_score = combo_5_score + 1
                    combo_5_cat += str_tmp

            list_combo_5_identical_bool.append(combo_5_score)
            list_combo_5_identical_cat.append(combo_5_cat)


            list_combo_6 = {'000000','111111','222222','333333','444444','555555','666666','777777','888888','999999'}
            combo_6_score = 0
            combo_6_cat = ""

            for str_tmp in list_combo_6:
                if str_tmp in list_number[i]:
                    combo_6_score = combo_6_score + 1
                    combo_6_cat += str_tmp

            list_combo_6_identical_bool.append(combo_6_score)
            list_combo_6_identical_cat.append(combo_6_cat)

    df['2_last'] = list_2_last
    df['3_last'] = list_3_last
    df['4_last'] = list_4_last
    df['5_last'] = list_5_last
    df['first'] = list_first

    df['3_combo_bool'] = list_combo_3_identical_bool
    df['3_combo_cat'] = list_combo_3_identical_cat

    df['4_combo_bool'] = list_combo_4_identical_bool
    df['4_combo_cat'] = list_combo_4_identical_cat

    df['5_combo_bool'] = list_combo_5_identical_bool
    df['5_combo_cat'] = list_combo_5_identical_cat

    df['6_combo_bool'] = list_combo_6_identical_bool
    df['6_combo_cat'] = list_combo_6_identical_cat

    df['3_combo_s'] = list_3_combo_s
    df['3_combo_57'] = list_3_combo_57
    df['3_combo_689'] = list_3_combo_689

    df['4_combo_s'] = list_4_combo_s
    df['4_combo_57'] = list_4_combo_57
    df['4_combo_689'] = list_4_combo_689
        
    return df

In [15]:
df = combo_feat(df)

In [16]:
df[df['3_combo_bool']!=0]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,5_combo_bool,5_combo_cat,6_combo_bool,6_combo_cat,3_combo_s,3_combo_57,3_combo_689,4_combo_s,4_combo_57,4_combo_689
196956,750893,sodepami_vn_0325036668,325036668,5000000,sodepami_v,9,3,2,5,0,...,0,,0,,0,0,1,0,0,0
561075,470974,sodepami_vn_0772861666,772861666,4000000,sodepami_v,9,7,7,2,8,...,0,,0,,0,0,1,0,0,0
735610,627687,sodepami_vn_0786111595,786111595,900000,sodepami_v,9,7,8,6,1,...,0,,0,,1,0,0,0,0,0
201696,867120,sodepami_vn_0777511991,777511991,1550000,sodepami_v,9,7,7,7,5,...,0,,0,,0,1,0,0,0,0
368659,503394,sodepami_vn_0704896999,704896999,5500000,sodepami_v,9,7,0,4,8,...,0,,0,,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32598,43724,sodepami_vn_0768444445,768444445,5800000,sodepami_v,9,7,6,8,4,...,1,44444,0,,1,0,0,1,0,0
554811,172837,sodepami_vn_02422482222,2422482222,2740000,sodepami_v,10,24,2,2,4,...,0,,0,,1,0,0,1,0,0
776833,847036,sodepami_vn_0784166688,784166688,4800000,sodepami_v,9,7,8,4,1,...,0,,0,,0,0,1,0,0,0
669835,655723,sodepami_vn_0888320585,888320585,800000,sodepami_v,9,8,8,8,3,...,0,,0,,0,0,1,0,0,0


In [17]:
# def lucky_2_last(df):
#     list_2_lucky_last_bool = list()
#     list_2_lucky_last_cat = list()

#     for i in list_2_last:
#         if (i == '68' or i == '86' or i == '78' or i == '79' or i == '39' or i == '38' or i =='83'):
#             list_2_lucky_last_bool.append(1)
#             list_2_lucky_last_cat.append(i)
#         else:
#             list_2_lucky_last_bool.append(0)
#             list_2_lucky_last_cat.append('')


#     df['2_lucky_last_bool'] = list_2_lucky_last_bool # check if the phone number has 2 last "lucky number" or not
#     df['2_lucky_last_cat'] = list_2_lucky_last_cat
    
#     return df


In [18]:
def sim_nam_sinh(df):
    list_nam_sinh_bool = []
    list_number = df['phone_number'].to_list()
    list_nam_sinh = []
    for i in list_number:
        if 1900 <= int(i[-4:]) <= 2022:
            list_nam_sinh.append(int(i[-4:]))
            list_nam_sinh_bool.append(1)
        else:
            list_nam_sinh.append(0)
            list_nam_sinh_bool.append(0)
            
    
    df['nam_sinh'] = list_nam_sinh
    df['nam_sinh_bool'] = list_nam_sinh_bool
    df['nam_sinh'] = df['nam_sinh'].astype(str)
    return df

In [19]:
df = sim_nam_sinh(df)

In [20]:
# df['nam_sinh'][df['nam_sinh_bool']!=0]

In [21]:
def symetric_4length_and_samesubtring_features(df):
    list_number = df['phone_number'].to_list()
    list_length_palindrome = []
    list_bool_palindrome = []
    list_palindrome = []
    list_same_substr = []
    list_same_substr_length = []
    list_bool_same_substr = []
    for i in range(len(list_number)):
        
        res = [list_number[i][x: y]+str(x) for x in range(len(list_number[i])-1)
              for y in range(x + 2, len(list_number[i])+1)]
                
        tmp = ""
        for j in res:
            if len(j[:-1])>=4  and len(j[:-1]) > len(tmp) and j[:-1] == j[:-1][::-1]:
                tmp = j[:-1]
        
         
        list_palindrome.append(tmp)
        list_length_palindrome.append(len(tmp))
        
        
        tmp2 = ""
                
                
        for j in range(len(res)-1):
            
            u1_n = (len(list_number[i]) - 1 - int(res[j][-1])) * len(res[j][:-1])
            ps = int(len(res[j][:-1]) * ( len(res[j][:-1])-1) / 2)
            
            
            for z in range(j + (u1_n - ps),len(res)):
                if res[j][:-1] == res[z][:-1] and len(res[j][:-1]) > len(tmp2):
                    tmp2 = res[z][:-1]
  
        list_same_substr.append(tmp2)
        list_same_substr_length.append(len(tmp2))
    
    for i in range(len(list_length_palindrome)):
        if list_length_palindrome[i] != 0:
            list_bool_palindrome.append(1)
        else:
            list_bool_palindrome.append(0)
        
        if list_same_substr_length[i] != 0:
            list_bool_same_substr.append(1)
        else:
            list_bool_same_substr.append(0)
        
    df['palindrome_length'] = list_length_palindrome
    df['palindrome'] = list_palindrome
    df['bool_palindrome'] = list_bool_palindrome
    df['list_same'] = list_same_substr
    df['list_same_substr_length'] = list_same_substr_length
    df['bool_substr'] = list_bool_same_substr
    return df

In [22]:
df = symetric_4length_and_samesubtring_features(df)
df.head(10)

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,4_combo_57,4_combo_689,nam_sinh,nam_sinh_bool,palindrome_length,palindrome,bool_palindrome,list_same,list_same_substr_length,bool_substr
818427,559367,sodepami_vn_0335984554,335984554,699000,sodepami_v,9,3,3,5,9,...,0,0,0,0,4,4554,1,,0,0
196956,750893,sodepami_vn_0325036668,325036668,5000000,sodepami_v,9,3,2,5,0,...,0,0,0,0,0,,0,,0,0
631141,998968,sodepami_vn_0987672989,987672989,1850000,sodepami_v,9,9,8,7,6,...,0,0,0,0,0,,0,98,2,1
304912,511809,sodepami_vn_0785863355,785863355,850000,sodepami_v,9,7,8,5,8,...,0,0,0,0,0,,0,,0,0
751212,439052,sodepami_vn_0984313012,984313012,1550000,sodepami_v,9,9,8,4,3,...,0,0,0,0,0,,0,,0,0
561075,470974,sodepami_vn_0772861666,772861666,4000000,sodepami_v,9,7,7,2,8,...,0,0,0,0,0,,0,,0,0
342615,1027522,sodepami_vn_0522015968,522015968,550000,sodepami_v,9,5,2,2,0,...,0,0,0,0,0,,0,,0,0
394965,847436,sodepami_vn_0342868909,342868909,800000,sodepami_v,9,3,4,2,8,...,0,0,0,0,0,,0,,0,0
818323,730000,sodepami_vn_0369781981,369781981,3000000,sodepami_v,9,3,6,9,7,...,0,0,1981,1,0,,0,81,2,1
179974,903858,sodepami_vn_0346240066,346240066,499000,sodepami_v,9,3,4,6,2,...,0,0,0,0,0,,0,,0,0


In [23]:
df[df['bool_substr']==1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,4_combo_57,4_combo_689,nam_sinh,nam_sinh_bool,palindrome_length,palindrome,bool_palindrome,list_same,list_same_substr_length,bool_substr
631141,998968,sodepami_vn_0987672989,987672989,1850000,sodepami_v,9,9,8,7,6,...,0,0,0,0,0,,0,98,2,1
818323,730000,sodepami_vn_0369781981,369781981,3000000,sodepami_v,9,3,6,9,7,...,0,0,1981,1,0,,0,81,2,1
559387,396083,sodepami_vn_0765056056,765056056,3000000,sodepami_v,9,7,6,5,0,...,0,0,0,0,5,65056,1,056,3,1
125770,482048,sodepami_vn_0878222290,878222290,3200000,sodepami_v,9,8,7,8,2,...,0,0,0,0,4,2222,1,22,2,1
884604,565231,sodepami_vn_0763013139,763013139,2220000,sodepami_v,9,7,6,3,0,...,0,0,0,0,0,,0,13,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363519,1042776,sodepami_vn_0784156515,784156515,700000,sodepami_v,9,7,8,4,1,...,0,0,0,0,5,15651,1,15,2,1
646591,461809,sodepami_vn_0384138373,384138373,650000,sodepami_v,9,3,8,4,1,...,0,0,0,0,0,,0,38,2,1
632732,428438,sodepami_vn_0816526526,816526526,8000000,sodepami_v,9,8,1,6,5,...,0,0,0,0,0,,0,652,3,1
230539,403416,sodepami_vn_0785031031,785031031,3500000,sodepami_v,9,7,8,5,0,...,0,0,0,0,0,,0,031,3,1


In [24]:
# def combo_features(df):
#     list_number = df['phone_number'].tolist()
    
#     list_2_last = list() #extract 2 last digit
    
#     list_3_last = list() #extract 3 last digit
    
#     list_4_last = list() #extract 4 last digit
    
#     list_5_last = list() #extract 5 last digit
    
#     list_combo_3_identical = list() # the phone number contain 111, 222, ....999 or not
    
#     list_combo_4_identical = list() # the phone number contain 1111, 2222, ... or not
    
#     list_combo_5_identical = list() # the phone number contain 11111, 22222, ... or not
    
#     list_combo_6_identical = list() # the phone number contain 11111, 22222, ... or not
    
    
    
#     for i in range(len(list_number)):
#         str_2_last = list_number[i][-2:]
#         list_2_last.append(str_2_last)
    
#         str_3_last = list_number[i][-3:]
#         list_3_last.append(str_3_last)
    
#         str_4_last = list_number[i][-4:]
#         list_4_last.append(str_4_last)
    
#         str_5_last = list_number[i][-5:]
#         list_5_last.append(str_5_last)
    
    
#         list_combo_3 = ['000','111','222','333','444','555','666','777','888','999']
    
#         tmp = ""
#         for j in range(len(list_combo_3)):
#             if list_combo_3[j] in list_number[i]:
#                 tmp += list_combo_3[j]
    
#         list_combo_3_identical.append(tmp)
    
#         list_combo_4 = ['0000','1111','2222','3333','4444','5555','6666','7777','8888','9999']
#         tmp_4 = ""
    
#         for j in range(len(list_combo_4)):
#           if list_combo_4[j] in list_number[i]:
#                 tmp_4 += list_combo_4[j]
        
#         list_combo_4_identical.append(tmp_4)
    
#         list_combo_5 = ['00000','11111','22222','33333','44444','55555','66666','77777','88888','99999']
#         tmp_5 = ""
    
#         for j in range(len(list_combo_5)):
#             if list_combo_5[j] in list_number[i]:
#                 tmp_5 += list_combo_5[j]
        
#         list_combo_5_identical.append(tmp_5)
    
    
#         list_combo_6 = ['000000','111111','222222','333333','444444','555555','666666','777777','888888','999999']
#         tmp_6 = ""
    
#         for j in range(len(list_combo_6)):
#             if list_combo_6[j] in list_number[i]:
#                 tmp_6 += list_combo_6[j]
        
#         list_combo_6_identical.append(tmp_6)
    
#     df['2_last'] = list_2_last
#     df['3_last'] = list_3_last
#     df['4_last'] = list_4_last
#     df['5_last'] = list_5_last
#     df['3_combo'] = list_combo_3_identical
#     df['4_combo'] = list_combo_4_identical
#     df['5_combo'] = list_combo_5_identical
#     df['6_combo'] = list_combo_6_identical
    
#     return df

In [25]:
# df = combo_features(df)
# df.head()

In [26]:
def count_combo_features(df):
    list_number = df['phone_number'].to_list()
    list_combo_3_count = list() # the phone number contain 111, 222, ....999 or not
    
    list_combo_4_count = list() # the phone number contain 1111, 2222, ... or not
    
    list_combo_5_count = list() # the phone number contain 11111, 22222, ... or not
    
    list_combo_6_count = list() # the phone number contain 11111, 22222, ... or not
    
    
    
    for i in range(len(list_number)):
    
        list_combo_3 = {'000','111','222','333','444','555','666','777','888','999'}
        combo_3_score = 0
    
        for j in list_combo_3:
            if j in list_number[i]:
                combo_3_score = combo_3_score + 1
    
        list_combo_3_count.append(combo_3_score)
    
        list_combo_4 = {'0000','1111','2222','3333','4444','5555','6666','7777','8888','9999'}
        combo_4_score = 0
    
        for j in list_combo_4:
            if j in list_number[i]:
                combo_4_score = combo_4_score + 1
        
        list_combo_4_count.append(combo_4_score)
    
        list_combo_5 = {'00000','11111','22222','33333','44444','55555','66666','77777','88888','99999'}
        combo_5_score = 0
    
        for j in list_combo_5:
            if j in list_number[i]:
                combo_5_score = combo_5_score + 1
        
        list_combo_5_count.append(combo_5_score)
    
    
        list_combo_6 = {'000000','111111','222222','333333','444444','555555','666666','777777','888888','999999'}
        combo_6_score = 0
    
        for j in list_combo_6:
            if j in list_number[i]:
                combo_6_score = combo_6_score + 1
        
        list_combo_6_count.append(combo_6_score)
        
    df['3_combo_count'] = list_combo_3_count
    df['4_combo_count'] = list_combo_4_count
    df['5_combo_count'] = list_combo_5_count
    df['6_combo_count'] = list_combo_6_count
    
    return df

In [27]:
df = count_combo_features(df)
df.head()

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,palindrome_length,palindrome,bool_palindrome,list_same,list_same_substr_length,bool_substr,3_combo_count,4_combo_count,5_combo_count,6_combo_count
818427,559367,sodepami_vn_0335984554,335984554,699000,sodepami_v,9,3,3,5,9,...,4,4554,1,,0,0,0,0,0,0
196956,750893,sodepami_vn_0325036668,325036668,5000000,sodepami_v,9,3,2,5,0,...,0,,0,,0,0,1,0,0,0
631141,998968,sodepami_vn_0987672989,987672989,1850000,sodepami_v,9,9,8,7,6,...,0,,0,98,2,1,0,0,0,0
304912,511809,sodepami_vn_0785863355,785863355,850000,sodepami_v,9,7,8,5,8,...,0,,0,,0,0,0,0,0,0
751212,439052,sodepami_vn_0984313012,984313012,1550000,sodepami_v,9,9,8,4,3,...,0,,0,,0,0,0,0,0,0


In [28]:
def features_2_inden_and_2_lucky_last(df):
    list_number = df['phone_number'].to_list()
    
    list_2_iden = list()
    
    list_2_combo = ['00','11','22','33','44','55','66','77','88','99']
    for i in list_number:
        tmp = ""
        for j in list_2_combo:
            if (j in i):
                tmp += j
            else:
                tmp += ""
                
        list_2_iden.append(tmp)
    
    list_2_lucky_last = list()
    
    list_lucky = ['68','86','78','79','39','38','83']
    for i in list_number:
      tmp = ""
      for j in list_lucky:
        if (j in i):
            tmp += j
        else:
            tmp += ""
    
      list_2_lucky_last.append(tmp)
    
    list_bool_2_lucky = []
    for i in list_2_lucky_last:
        if i != "":
            list_bool_2_lucky.append(1)
        else:
            list_bool_2_lucky.append(0)
    
    list_bool_2_iden = []
    for i in list_2_iden:
        if i != "":
            list_bool_2_iden.append(1)
        else:
            list_bool_2_iden.append(0)
    
    df['2_lucky_bool'] = list_bool_2_lucky
    df['2_iden_bool'] = list_bool_2_iden
    df['2_iden'] = list_2_iden
    df['2_lucky'] = list_2_lucky_last
    return df

In [29]:
df = features_2_inden_and_2_lucky_last(df)
df.head(8)

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,list_same_substr_length,bool_substr,3_combo_count,4_combo_count,5_combo_count,6_combo_count,2_lucky_bool,2_iden_bool,2_iden,2_lucky
818427,559367,sodepami_vn_0335984554,335984554,699000,sodepami_v,9,3,3,5,9,...,0,0,0,0,0,0,0,1,3355,
196956,750893,sodepami_vn_0325036668,325036668,5000000,sodepami_v,9,3,2,5,0,...,0,0,1,0,0,0,1,1,66,68
631141,998968,sodepami_vn_0987672989,987672989,1850000,sodepami_v,9,9,8,7,6,...,2,1,0,0,0,0,0,0,,
304912,511809,sodepami_vn_0785863355,785863355,850000,sodepami_v,9,7,8,5,8,...,0,0,0,0,0,0,1,1,3355,8678
751212,439052,sodepami_vn_0984313012,984313012,1550000,sodepami_v,9,9,8,4,3,...,0,0,0,0,0,0,0,0,,
561075,470974,sodepami_vn_0772861666,772861666,4000000,sodepami_v,9,7,7,2,8,...,0,0,1,0,0,0,1,1,6677,86
342615,1027522,sodepami_vn_0522015968,522015968,550000,sodepami_v,9,5,2,2,0,...,0,0,0,0,0,0,1,1,22,68
394965,847436,sodepami_vn_0342868909,342868909,800000,sodepami_v,9,3,4,2,8,...,0,0,0,0,0,0,1,0,,6886


In [30]:
df[df['2_iden_bool']==1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,list_same_substr_length,bool_substr,3_combo_count,4_combo_count,5_combo_count,6_combo_count,2_lucky_bool,2_iden_bool,2_iden,2_lucky
818427,559367,sodepami_vn_0335984554,335984554,699000,sodepami_v,9,3,3,5,9,...,0,0,0,0,0,0,0,1,3355,
196956,750893,sodepami_vn_0325036668,325036668,5000000,sodepami_v,9,3,2,5,0,...,0,0,1,0,0,0,1,1,66,68
304912,511809,sodepami_vn_0785863355,785863355,850000,sodepami_v,9,7,8,5,8,...,0,0,0,0,0,0,1,1,3355,8678
561075,470974,sodepami_vn_0772861666,772861666,4000000,sodepami_v,9,7,7,2,8,...,0,0,1,0,0,0,1,1,6677,86
342615,1027522,sodepami_vn_0522015968,522015968,550000,sodepami_v,9,5,2,2,0,...,0,0,0,0,0,0,1,1,22,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471157,1061915,sodepami_vn_0567233768,567233768,700000,sodepami_v,9,5,6,7,2,...,0,0,0,0,0,0,1,1,33,68
301924,143946,sodepami_vn_0355939779,355939779,3630000,sodepami_v,9,3,5,5,9,...,0,0,0,0,0,0,1,1,5577,7939
430799,382141,sodepami_vn_0794462555,794462555,1100000,sodepami_v,9,7,9,4,4,...,0,0,1,0,0,0,1,1,4455,79
815988,185727,sodepami_vn_0938592252,938592252,1850000,sodepami_v,9,9,3,8,5,...,0,0,0,0,0,0,1,1,22,38


In [31]:
def last_special_case(df):
    
    list_3_last = df['3_last'].to_list()
    list_4_last = df['4_last'].to_list()
    list_5_last = df['5_last'].to_list()
    
    list_3_last_iden = list()
    
    list_3_last_bool = []
    list_4_last_bool = []
    list_5_last_bool = []
    for x in list_3_last:
        if (x == '000' or x == '111' or x == '222' or x == '333' or x == '444' or x == '555' or x == '666' or x == '777' or x == '888' or x == '999'):
            list_3_last_iden.append(x)
            list_3_last_bool.append(1)
        else:
            list_3_last_iden.append("n")
            list_3_last_bool.append(0)
    
    
    df['3_last_iden'] = list_3_last_iden # if the phone number have 3 last number indentical or not
    df['3_last_iden_bool'] = list_3_last_bool
    
    list_4_last_iden = list()
    
    for x in list_4_last:
        if (x == '0000' or x == '1111' or x == '2222' or x == '3333' or x == '4444' or x == '5555' or x == '6666' or x == '7777' or x == '8888' or x == '9999'):
            list_4_last_iden.append(x)
            list_4_last_bool.append(1)
        else:
            list_4_last_iden.append("n")
            list_4_last_bool.append(0)
    
    
    df['4_last_iden'] = list_4_last_iden # if the phone number have 4 last number indentical or not
    df['4_last_iden_bool'] = list_4_last_bool
    
    list_5_last_iden = list()
    
    for x in list_5_last:
        if (x == '00000' or x == '11111' or x == '22222' or x == '33333' or x == '44444' or x == '55555' or x == '66666' or x == '77777' or x == '88888' or x == '99999'):
            list_5_last_iden.append(x)
            list_5_last_bool.append(1)
        else:
            list_5_last_iden.append("n")
            list_5_last_bool.append(0)
    
    
    df['5_last_iden'] = list_5_last_iden # if the phone number have 4 last number indentical or not
    df['5_last_iden_bool'] = list_5_last_bool
    
    list_3_last_increasing_bool = []
    list_4_last_increasing_bool = []
    list_5_last_increasing_bool = []
    
    list_3_last_increasing_sample = ('012','123','234','345','456','567','678','789','024','135','246','357','468','579','036','147','258','369','048','159')
    list_3_last_increasing = list()
    
    for x in list_3_last:
      check = 0
      for j in list_3_last_increasing_sample:
        if j in x:
            list_3_last_increasing.append(x)
            check = 1
            list_3_last_increasing_bool.append(1)
            break
      if check == 0:
        list_3_last_increasing.append("n") # 3-last increasing number
        list_3_last_increasing_bool.append(0)
    
    df['3_last_increasing'] = list_3_last_increasing
    df['3_last_increasing_bool'] = list_3_last_increasing_bool
    
    list_4_last_increasing_sample = ('0123','1234','2345','3456','4567','5678','6789','0246','1357','2468','3579','0369')
    list_4_last_increasing = list()
    
    for x in list_4_last:
      check = 0
      for j in list_4_last_increasing_sample:
        if j in x:
            list_4_last_increasing.append(x)
            check = 1
            list_4_last_increasing_bool.append(1)
            break
        
      if check == 0:
        list_4_last_increasing.append("n") # 4-last increasing number 
        list_4_last_increasing_bool.append(0)
    
    df['4_last_increasing'] = list_4_last_increasing
    df['4_last_increasing_bool'] = list_4_last_increasing_bool
    
    list_5_last_increasing_sample = ('01234','12345','23456','34567','45678','56789','02468','13579')
    list_5_last_increasing = list()
    
    for x in list_5_last:
      check = 0
      for j in list_5_last_increasing_sample:
        if j in x:
            list_5_last_increasing.append(x)
            check =1
            list_5_last_increasing_bool.append(1)
            break
      if check == 0:      
        list_5_last_increasing.append("n") # 3-last increasing number 
        list_5_last_increasing_bool.append(0)
    
    df['5_last_increasing'] = list_5_last_increasing
    df['5_last_increasing_bool'] = list_5_last_increasing_bool
    
    return df

In [32]:
df = last_special_case(df)
df.sample(10)

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,4_last_iden,4_last_iden_bool,5_last_iden,5_last_iden_bool,3_last_increasing,3_last_increasing_bool,4_last_increasing,4_last_increasing_bool,5_last_increasing,5_last_increasing_bool
28152,894845,sodepami_vn_0866017919,866017919,900000,sodepami_v,9,8,6,6,0,...,n,0,n,0,n,0,n,0,n,0
664378,869559,sodepami_vn_0816691456,816691456,1000000,sodepami_v,9,8,1,6,6,...,n,0,n,0,456,1,n,0,n,0
331225,626009,sodepami_vn_0925983789,925983789,1400000,sodepami_v,9,9,2,5,9,...,n,0,n,0,789,1,n,0,n,0
436222,821772,sodepami_vn_0589744579,589744579,700000,sodepami_v,9,5,8,9,7,...,n,0,n,0,579,1,n,0,n,0
550464,799230,sodepami_vn_0769351989,769351989,2550000,sodepami_v,9,7,6,9,3,...,n,0,n,0,n,0,n,0,n,0
588866,318173,sodepami_vn_0815475544,815475544,900000,sodepami_v,9,8,1,5,4,...,n,0,n,0,n,0,n,0,n,0
366050,876135,sodepami_vn_0789920368,789920368,700000,sodepami_v,9,7,8,9,9,...,n,0,n,0,n,0,n,0,n,0
723669,47780,sodepami_vn_0782484444,782484444,8000000,sodepami_v,9,7,8,2,4,...,4444,1,n,0,n,0,n,0,n,0
228742,726650,sodepami_vn_0888105579,888105579,1100000,sodepami_v,9,8,8,8,1,...,n,0,n,0,579,1,n,0,n,0
861499,294893,sodepami_vn_0981911176,981911176,650000,sodepami_v,9,9,8,1,9,...,n,0,n,0,n,0,n,0,n,0


In [33]:
def features_from_sum(df):
    list_number = df['phone_number'].to_list()
    list_sum = []
    for i in range(len(list_number)):
      tmp = 0
      for j in range(len(list_number[i])):
        tmp += int(list_number[i][j])
      list_sum.append(tmp)
    
    list_phong_thuy = []
    
    for i in range(len(list_sum)):
      list_phong_thuy.append(list_sum[i]%10)
    
    df['phong_thuy_1'] = list_phong_thuy
    
    list_phong_thuy_2 = []
    
    for i in range(len(list_sum)):
      k = list_sum[i]
      ans = 0
      while(k>=0):
        ans+=k%10
        k = math.floor(k/10)
        if (k==0 and ans >= 10):
          k = ans
          ans = 0
        if (k==0 and ans < 10):
          list_phong_thuy_2.append(ans)
          break
      
    df['phong_thuy_2'] = list_phong_thuy_2
    
    return df

In [34]:
df = features_from_sum(df)
df.shape

(628984, 75)

In [35]:
def creating_label(df):
    price = np.array(df['price_vnd'])
    label = []

    for i in range(df.shape[0]):
      if price[i] <= 1.6e6:
        label.append(0)
        continue
      if price[i] <= 3.6e6:
        label.append(1)
        continue
      if price[i] <= 6.5e6:
        label.append(2)
        continue
      if price[i] <= 10.8e6:
        label.append(3)
        continue
      if price[i] <= 1.5e7:
        label.append(4)
        continue
      if price[i] <= 6.1e7:
        label.append(5)
        continue
      if price[i] <= 1.5e8:
        label.append(6)
        continue
      if price[i] <= 3.1e8:
        label.append(7)
        continue
      if price[i] <= 5.3e8:
        label.append(8)
        continue
  
      label.append(9)

    df['label'] = label
    
    return df


In [36]:
df = creating_label(df)
df.head(10)

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,5_last_iden_bool,3_last_increasing,3_last_increasing_bool,4_last_increasing,4_last_increasing_bool,5_last_increasing,5_last_increasing_bool,phong_thuy_1,phong_thuy_2,label
818427,559367,sodepami_vn_0335984554,335984554,699000,sodepami_v,9,3,3,5,9,...,0,n,0,n,0,n,0,6,1,0
196956,750893,sodepami_vn_0325036668,325036668,5000000,sodepami_v,9,3,2,5,0,...,0,n,0,n,0,n,0,9,3,2
631141,998968,sodepami_vn_0987672989,987672989,1850000,sodepami_v,9,9,8,7,6,...,0,n,0,n,0,n,0,5,2,1
304912,511809,sodepami_vn_0785863355,785863355,850000,sodepami_v,9,7,8,5,8,...,0,n,0,n,0,n,0,0,5,0
751212,439052,sodepami_vn_0984313012,984313012,1550000,sodepami_v,9,9,8,4,3,...,0,012,1,n,0,n,0,1,4,0
561075,470974,sodepami_vn_0772861666,772861666,4000000,sodepami_v,9,7,7,2,8,...,0,n,0,n,0,n,0,9,4,2
342615,1027522,sodepami_vn_0522015968,522015968,550000,sodepami_v,9,5,2,2,0,...,0,n,0,n,0,n,0,8,2,0
394965,847436,sodepami_vn_0342868909,342868909,800000,sodepami_v,9,3,4,2,8,...,0,n,0,n,0,n,0,9,4,0
818323,730000,sodepami_vn_0369781981,369781981,3000000,sodepami_v,9,3,6,9,7,...,0,n,0,n,0,n,0,2,7,1
179974,903858,sodepami_vn_0346240066,346240066,499000,sodepami_v,9,3,4,6,2,...,0,n,0,n,0,n,0,1,4,0


In [37]:
def creating_features_by_combine_pos(df):
    for i in range(1,9):
        for j in range(i+1,10):
            df['pos_'+str(i)+"_"+str(j)] = df['pos_' + str(i)] + df['pos_' + str(j)]
    
    for i in range(1,8):
        for j in range(i+1,9):
            for k in range(j+1,10):
                df['pos_'+str(i)+"_"+str(j)+"_"+str(k)] = df['pos_' + str(i)] + df['pos_' + str(j)] + df['pos_'+str(k)]
      
#     for i in range(1,7):
#         for j in range(i+1,8):
#             for k in range(j+1,9):
#                 for l in range(k+1,10):
#                     #df_tmp = pd.DataFrame()
#                     df['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l)] = df['pos_' + str(i)] \
#                     + df['pos_' + str(j)] + df['pos_'+str(k)] + df['pos_'+str(l)]
#                     df_ans = pd.concat([df_ans, df_tmp],axis = 1)
                
#     df['pos_1_2_8_9'] = df['pos_1'] + df['pos_2'] + df['pos_8'] + df['pos_9']
    
#     for i in range(1,7):
#         df['pos_'+str(i)+"_"+str(i+1)+"_"+str(i+2)+"_"+str(i+3)] = df['pos_' + str(i)] + df['pos_' + str(i+1)] + df['pos_'+str(i+2)] + df['pos_' + str(i+3)]
    
    return df
    
# def creating_features_by_combine_pos(df):
#     df_ans = df
    
# #     for i in range(1,10):
# #         df['pos_' + str(i)] = df['pos_'+str(i)].astype(str)
        
#     for i in range(1,7):
#         for j in range(i+1,8):
#             for k in range(j+1,9):
#                 for l in range(k+1,10):
#                     df_tmp = pd.DataFrame()
#                     df_tmp['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l)] = df['pos_' + str(i)] \
#                     + df['pos_' + str(j)] + df['pos_'+str(k)] + df['pos_'+str(l)]
#                     df_ans = pd.concat([df_ans, df_tmp],axis = 1)
    
    
#     return df_ans

In [38]:
df = creating_features_by_combine_pos(df)
df.shape

(628984, 196)

In [39]:
# df['pos_1_2_3_4'].value_counts()

### Tuan feats

In [40]:
def sim_loc_phat(df):
    list_number = df['phone_number'].to_list()
    list_2_last = list() #extract 2 last digit

    list_4_last = list() #extract 3 last digit
    
    for i in range(len(list_number)):
        str_tmp_2_last = list_number[i][-2:]
        list_2_last.append(str_tmp_2_last)
    
        str_tmp_4_last = list_number[i][-4:]
        list_4_last.append(str_tmp_4_last)
        
        
    list_phat_loc_2_bool = []
    list_phat_loc_2_cat = []

    list_phat_loc_4_bool = []
    list_phat_loc_4_cat = []

    list_phat_loc_2_sample = ['68','86']
    list_phat_loc_4_sample = ['6688','6868','6886', '8668']

    for x in list_2_last:
        if list_phat_loc_2_sample.count(x)>0:
            list_phat_loc_2_bool.append(1)
            list_phat_loc_2_cat.append(x)
        else:
            list_phat_loc_2_bool.append(0)
            list_phat_loc_2_cat.append('')
        

    for x in list_4_last:
        if list_phat_loc_4_sample.count(x)>0:
            list_phat_loc_4_bool.append(1)
            list_phat_loc_4_cat.append(x)
        else:
            list_phat_loc_4_bool.append(0)
            list_phat_loc_4_cat.append('')      
        
    df['phat_loc_2_bool'] = list_phat_loc_2_bool
    df['phat_loc_2_cat'] = list_phat_loc_2_cat

    df['phat_loc_4_bool'] = list_phat_loc_4_bool
    df['phat_loc_4_cat'] = list_phat_loc_4_cat
    
    return df


In [41]:
df = sim_loc_phat(df)

In [42]:
df[df['phat_loc_2_bool'] == 1].shape

(93188, 200)

In [43]:
def increasing_3_4(df):
    list_number = df['phone_number'].to_list()
    list_double_3_increasing = []
    
    list_3_last_increasing_bool = df['3_last_increasing_bool'].to_list()
    list_4_last_increasing_bool = df['4_last_increasing_bool'].to_list()
    for i in range(len(list_number)):
        if (list_3_last_increasing_bool[i] == 1):
            if (list_number[i][3:6] == list_number[i][6:9]):
                list_double_3_increasing.append(1)
            else: list_double_3_increasing.append(0)
        else: list_double_3_increasing.append(0)

    df['list_double_3_increasing']= list_double_3_increasing
    
    list_double_4_increasing = []

    for i in range(len(list_number)):
        if (list_4_last_increasing_bool[i] == 1):
            if (list_number[i][1:5] == list_number[i][5:9]):
                list_double_4_increasing.append(1)
            else: list_double_4_increasing.append(0)
        else: list_double_4_increasing.append(0)
    
    df['list_double_4_increasing']= list_double_4_increasing
    
    return df
        

In [44]:
df = increasing_3_4(df)

In [45]:
def double_3_last(df):
    list_number = df['phone_number'].to_list()
    list_double_3_last = []

    for i in range(len(list_number)):
        if (list_number[i][3:6] == list_number[i][6:9]):
            list_double_3_last.append(1)
        else: list_double_3_last.append(0)
    
    df['list_double_3_last']= list_double_3_last
    return df

In [46]:
df = double_3_last(df)

In [47]:
df[df['list_double_3_last']==1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,pos_6_7_9,pos_6_8_9,pos_7_8_9,phat_loc_2_bool,phat_loc_2_cat,phat_loc_4_bool,phat_loc_4_cat,list_double_3_increasing,list_double_4_increasing,list_double_3_last
559387,396083,sodepami_vn_0765056056,765056056,3000000,sodepami_v,9,7,6,5,0,...,606,656,056,0,,0,,0,0,1
17849,52200,sodepami_vn_0562493493,562493493,640000,sodepami_v,9,5,6,2,4,...,343,393,493,0,,0,,0,0,1
358590,125588,sodepami_vn_0563302302,563302302,650000,sodepami_v,9,5,6,3,3,...,232,202,302,0,,0,,0,0,1
65932,220753,sodepami_vn_0842988988,842988988,29000000,sodepami_v,9,8,4,2,9,...,898,888,988,0,,0,,0,0,1
192346,258801,sodepami_vn_0769209209,769209209,4500000,sodepami_v,9,7,6,9,2,...,929,909,209,0,,0,,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751148,46654,sodepami_vn_0566490490,566490490,640000,sodepami_v,9,5,6,6,4,...,040,090,490,0,,0,,0,0,1
700894,357764,sodepami_vn_0797631631,797631631,8800000,sodepami_v,9,7,9,7,6,...,161,131,631,0,,0,,0,0,1
113254,193139,sodepami_vn_0776512512,776512512,5500000,sodepami_v,9,7,7,6,5,...,252,212,512,0,,0,,0,0,1
632732,428438,sodepami_vn_0816526526,816526526,8000000,sodepami_v,9,8,1,6,5,...,656,626,526,0,,0,,0,0,1


In [48]:
def triple_2_last(df):
    list_number = df['phone_number'].to_list()
    list_triple_2_last_r1 = [] # list for triple 2 last but too not 'beautiful'
    tmp_list = [5,6,8,9]

    for i in range(len(list_number)):
        if(int(list_number[i][8]) in tmp_list and int(list_number[i][7]) in tmp_list ):
            list_triple_2_last_r1.append(0)
        else:
            if (list_number[i][3:5] == list_number[i][5:7] == list_number[i][7:9]):
                list_triple_2_last_r1.append(1)
            else:
                list_triple_2_last_r1.append(0)
                
    df['triple_2_last_r1'] = list_triple_2_last_r1
    return df

In [49]:

df = triple_2_last(df)

In [50]:
df[df['triple_2_last_r1'] == 1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,pos_6_8_9,pos_7_8_9,phat_loc_2_bool,phat_loc_2_cat,phat_loc_4_bool,phat_loc_4_cat,list_double_3_increasing,list_double_4_increasing,list_double_3_last,triple_2_last_r1
725117,49590,sodepami_vn_0559636363,559636363,35800000,sodepami_v,9,5,5,9,6,...,663,363,0,,0,,0,0,0,1
263232,32277,sodepami_vn_0564323232,564323232,16000000,sodepami_v,9,5,6,4,3,...,332,232,0,,0,,0,0,0,1
273506,45049,sodepami_vn_0352878787,352878787,45000000,sodepami_v,9,3,5,2,8,...,887,787,0,,0,,0,0,0,1
333677,15135,sodepami_vn_0926838383,926838383,230000000,sodepami_v,9,9,2,6,8,...,883,383,0,,0,,0,0,0,1
412911,66527,sodepami_vn_0352747474,352747474,23000000,sodepami_v,9,3,5,2,7,...,774,474,0,,0,,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67567,15932,sodepami_vn_0935626262,935626262,328000000,sodepami_v,9,9,3,5,6,...,662,262,0,,0,,0,0,0,1
384943,59961,sodepami_vn_0769424242,769424242,28000000,sodepami_v,9,7,6,9,4,...,442,242,0,,0,,0,0,0,1
590685,22453,sodepami_vn_0776878787,776878787,45000000,sodepami_v,9,7,7,6,8,...,887,787,0,,0,,0,0,0,1
84984,51882,sodepami_vn_0587232323,587232323,18000000,sodepami_v,9,5,8,7,2,...,223,323,0,,0,,0,0,0,1


In [51]:
# Sim kep duoi (ABB.CDD) va sim lap dau(ABC.ABD), lap cuoi (ABC.DBC)
def sim_6_last(df):
    list_number = df['phone_number'].to_list()
    list_kep_duoi_bool = []
    list_kep_duoi_cat = []

    list_lap_dau_bool = []
    list_lap_dau_cat = []

    list_lap_cuoi_bool = []
    list_lap_cuoi_cat = []

    for x in list_number:
        if (x[4:6] == x[7:]):
            list_lap_cuoi_bool.append(1)
            list_lap_cuoi_cat.append(x[4:6])
        
            if (x[7] == x[8]):
                list_kep_duoi_bool.append(1)
                list_kep_duoi_cat.append(x[8])
            else:
                list_kep_duoi_bool.append(0)
                list_kep_duoi_cat.append('')
        else:
            list_lap_cuoi_bool.append(0)
            list_lap_cuoi_cat.append('')
        
            list_kep_duoi_bool.append(0)
            list_kep_duoi_cat.append('')
        
        if (x[3:5] == x[6:8]):
            list_lap_dau_bool.append(1)
            list_lap_dau_cat.append(x[3:5])
        else:
            list_lap_dau_bool.append(0)
            list_lap_dau_cat.append('')
    df['kep_duoi_bool'] = list_kep_duoi_bool
    df['kep_duoi_cat'] = list_kep_duoi_cat

    df['lap_dau_bool'] = list_lap_dau_bool
    df['lap_dau_cat'] = list_lap_dau_cat

    df['lap_cuoi_bool'] = list_lap_cuoi_bool
    df['lap_cuoi_cat'] = list_lap_cuoi_cat
    
    return df

In [52]:

df = sim_6_last(df)

In [53]:
df[df['lap_dau_bool']==1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,list_double_3_increasing,list_double_4_increasing,list_double_3_last,triple_2_last_r1,kep_duoi_bool,kep_duoi_cat,lap_dau_bool,lap_dau_cat,lap_cuoi_bool,lap_cuoi_cat
559387,396083,sodepami_vn_0765056056,765056056,3000000,sodepami_v,9,7,6,5,0,...,0,0,1,0,0,,1,05,1,56
17849,52200,sodepami_vn_0562493493,562493493,640000,sodepami_v,9,5,6,2,4,...,0,0,1,0,0,,1,49,1,93
536833,144998,sodepami_vn_0854723724,854723724,700000,sodepami_v,9,8,5,4,7,...,0,0,0,0,0,,1,72,0,
358590,125588,sodepami_vn_0563302302,563302302,650000,sodepami_v,9,5,6,3,3,...,0,0,1,0,0,,1,30,1,02
542772,710823,sodepami_vn_0765463468,765463468,1000000,sodepami_v,9,7,6,5,4,...,0,0,0,0,0,,1,46,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113254,193139,sodepami_vn_0776512512,776512512,5500000,sodepami_v,9,7,7,6,5,...,0,0,1,0,0,,1,51,1,12
32598,43724,sodepami_vn_0768444445,768444445,5800000,sodepami_v,9,7,6,8,4,...,0,0,0,0,0,,1,44,0,
697494,124671,sodepami_vn_0843765766,843765766,1100000,sodepami_v,9,8,4,3,7,...,0,0,0,0,0,,1,76,0,
632732,428438,sodepami_vn_0816526526,816526526,8000000,sodepami_v,9,8,1,6,5,...,0,0,1,0,0,,1,52,1,26


In [54]:
def special_4_sim(df):
    list_number = df['phone_number'].to_list()
    list_2_lap_t1 = [] #ABAB, A>B
    list_2_lap_t2 = [] # ABAB, A<B

    list_2_lap_t3 = [] # AABB, A>B
    list_2_lap_t4 = [] # AABB, A<B

    for i in range(len(list_number)):
        if (list_number[i][5:7] == list_number[i][7:]):
            if (int(list_number[i][7]) < int(list_number[i][8])):
                list_2_lap_t2.append(1)
                list_2_lap_t1.append(0)
            elif(int(list_number[i][7]) > int(list_number[i][8])):
                list_2_lap_t1.append(1)
                list_2_lap_t2.append(0)
            else:
                list_2_lap_t1.append(0)
                list_2_lap_t2.append(0)

        else:
            list_2_lap_t1.append(0)
            list_2_lap_t2.append(0)


        if (list_number[i][5]==list_number[i][6] and list_number[i][7]==list_number[i][8]):
            if(list_number[i][5] < list_number[i][7]):
                list_2_lap_t4.append(1)
                list_2_lap_t3.append(0)
            elif(list_number[i][5] > list_number[i][7]):
                list_2_lap_t4.append(0)
                list_2_lap_t3.append(1)
            else:
                list_2_lap_t4.append(0)
                list_2_lap_t3.append(0)

        else:
            list_2_lap_t4.append(0)
            list_2_lap_t3.append(0)
    df['2_lap_t1'] = list_2_lap_t1
    df['2_lap_t2'] = list_2_lap_t2
    df['2_lap_t3'] = list_2_lap_t3
    df['2_lap_t4'] = list_2_lap_t4
    
    return df

In [55]:

df = special_4_sim(df)


In [56]:
df[df['2_lap_t4']==1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,kep_duoi_bool,kep_duoi_cat,lap_dau_bool,lap_dau_cat,lap_cuoi_bool,lap_cuoi_cat,2_lap_t1,2_lap_t2,2_lap_t3,2_lap_t4
304912,511809,sodepami_vn_0785863355,785863355,850000,sodepami_v,9,7,8,5,8,...,0,,0,,0,,0,0,0,1
179974,903858,sodepami_vn_0346240066,346240066,499000,sodepami_v,9,3,4,6,2,...,0,,0,,0,,0,0,0,1
237509,233245,sodepami_vn_0818005599,818005599,15000000,sodepami_v,9,8,1,8,0,...,0,,0,,0,,0,0,0,1
848866,489934,sodepami_vn_0363444466,363444466,4500000,sodepami_v,9,3,6,3,4,...,0,,0,,0,,0,0,0,1
504658,755347,sodepami_vn_0839388899,839388899,15000000,sodepami_v,9,8,3,9,3,...,0,,0,,0,,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625912,506527,sodepami_vn_0799935577,799935577,1550000,sodepami_v,9,7,9,9,9,...,0,,0,,0,,0,0,0,1
176970,468762,sodepami_vn_0829673399,829673399,800000,sodepami_v,9,8,2,9,6,...,0,,0,,0,,0,0,0,1
776833,847036,sodepami_vn_0784166688,784166688,4800000,sodepami_v,9,7,8,4,1,...,0,,0,,0,,0,0,0,1
269446,244251,sodepami_vn_0812441199,812441199,2050000,sodepami_v,9,8,1,2,4,...,0,,0,,0,,0,0,0,1


In [57]:
def ong_dia(df):
    list_number = df['phone_number'].to_list()
    list_4_last = []
    for i in range(len(list_number)):
        str_tmp_4_last = list_number[i][-4:]
        list_4_last.append(str_tmp_4_last)
    
    list_lap_ong_dia = [] #7997, 8998
    list_lap_ong_dia_last = []
    list_lap_ong_dia_sample = ['7997', '8998']

    list_kep_ong_dia = [] #7799, 8899
    list_kep_ong_dia_last = []
    list_kep_ong_dia_sample = ['7799', '8899']

    list_dao_ong_dia_1 = [] #9779, 9889
    list_dao_ong_dia_1_last = []
    list_dao_ong_dia_1_sample = ['9779', '9889']

    list_dao_ong_dia_2 = [] #7997, 8998
    list_dao_ong_dia_2_last = []
    list_dao_ong_dia_2_sample = ['7997', '8998']

    for i in range(len(list_number)):
        #lap ong dia
        lap_ong_dia_score = 0
        lap_ong_dia_last_score = 0
        for tmp_str in list_lap_ong_dia_sample:
            if tmp_str in list_number[i]:
                lap_ong_dia_score = lap_ong_dia_score +1

            if (tmp_str == list_4_last[i]):
                lap_ong_dia_last_score = lap_ong_dia_last_score +1

        list_lap_ong_dia.append(lap_ong_dia_score)
        list_lap_ong_dia_last.append(lap_ong_dia_last_score)

        #kep ong dia
        kep_ong_dia_score = 0
        kep_ong_dia_last_score = 0
        for tmp_str in list_kep_ong_dia_sample:
            if tmp_str in list_number[i]:
                kep_ong_dia_score = kep_ong_dia_score +1

            if (tmp_str == list_4_last[i]):
                kep_ong_dia_last_score = kep_ong_dia_last_score +1

        list_kep_ong_dia.append(kep_ong_dia_score)
        list_kep_ong_dia_last.append(kep_ong_dia_last_score)   

        #dao ong dia 1
        dao_ong_dia_1_score = 0
        dao_ong_dia_1_last_score = 0
        for tmp_str in list_dao_ong_dia_1_sample:
            if tmp_str in list_number[i]:
                dao_ong_dia_1_score = dao_ong_dia_1_score +1

            if (tmp_str == list_4_last[i]):
                dao_ong_dia_1_last_score = dao_ong_dia_1_last_score +1

        list_dao_ong_dia_1.append(dao_ong_dia_1_score)
        list_dao_ong_dia_1_last.append(dao_ong_dia_1_last_score)   

        #dao ong dia 22
        dao_ong_dia_2_score = 0
        dao_ong_dia_2_last_score = 0
        for tmp_str in list_dao_ong_dia_2_sample:
            if tmp_str in list_number[i]:
                dao_ong_dia_2_score = dao_ong_dia_2_score +1

            if (tmp_str == list_4_last[i]):
                dao_ong_dia_2_last_score = dao_ong_dia_2_last_score +1

        list_dao_ong_dia_2.append(dao_ong_dia_2_score)
        list_dao_ong_dia_2_last.append(dao_ong_dia_2_last_score)  
    
    df['lap_ong_dia'] = list_lap_ong_dia 
    df['lap_ong_dia_last'] = list_lap_ong_dia_last 
    df['kep_ong_dia'] = list_kep_ong_dia 
    df['kep_ong_dia_last'] = list_kep_ong_dia_last 
    df['dao_ong_dia_1'] = list_dao_ong_dia_1
    df['dao_ong_dia_1_last'] = list_dao_ong_dia_1_last 
    df['dao_ong_dia_2'] = list_dao_ong_dia_2
    df['dao_ong_dia_2_last'] = list_dao_ong_dia_2_last
    
    return df


In [58]:

df = ong_dia(df)

In [59]:
df[df['dao_ong_dia_2_last']==1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,2_lap_t3,2_lap_t4,lap_ong_dia,lap_ong_dia_last,kep_ong_dia,kep_ong_dia_last,dao_ong_dia_1,dao_ong_dia_1_last,dao_ong_dia_2,dao_ong_dia_2_last
116070,234435,sodepami_vn_0929608998,929608998,1285000,sodepami_v,9,9,2,9,6,...,0,0,1,1,0,0,0,0,1,1
776188,574010,sodepami_vn_0886218998,886218998,2050000,sodepami_v,9,8,8,6,2,...,0,0,1,1,0,0,0,0,1,1
243567,681007,sodepami_vn_0372297997,372297997,1550000,sodepami_v,9,3,7,2,2,...,0,0,1,1,0,0,0,0,1,1
267924,570409,sodepami_vn_0917148998,917148998,2050000,sodepami_v,9,9,1,7,1,...,0,0,1,1,0,0,0,0,1,1
162278,465569,sodepami_vn_0372008998,372008998,2700000,sodepami_v,9,3,7,2,0,...,0,0,1,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522415,776972,sodepami_vn_0939658998,939658998,5800000,sodepami_v,9,9,3,9,6,...,0,0,1,1,0,0,0,0,1,1
782485,436377,sodepami_vn_0877997997,877997997,20000000,sodepami_v,9,8,7,7,9,...,0,0,1,1,1,0,0,0,1,1
776946,299182,sodepami_vn_0983648998,983648998,6800000,sodepami_v,9,9,8,3,6,...,0,0,1,1,0,0,0,0,1,1
629335,701609,sodepami_vn_0929207997,929207997,1100000,sodepami_v,9,9,2,9,2,...,0,0,1,1,0,0,0,0,1,1


In [60]:
def special_6_last(df):
    list_number = df['phone_number'].to_list()
    list_ABBBBA_1 = [] #A<B
    list_ABBBBA_2 = [] #A>B

    list_ABBA_1 = [] #A<B
    list_ABBA_2 = [] #A>B

    for i in range(len(list_number)):
        if (list_number[i][3] == list_number[i][8] and list_number[i][4] == list_number[i][5] and list_number[i][4] == list_number[i][6] and list_number[i][4] == list_number[i][7]):
            if (int(list_number[i][3]) < int(list_number[i][4])):
                list_ABBBBA_1.append(1)
                list_ABBBBA_2.append(0)
            elif(int(list_number[i][3]) > int(list_number[i][4])):
                list_ABBBBA_1.append(0)
                list_ABBBBA_2.append(1)
            else:
                list_ABBBBA_1.append(0)
                list_ABBBBA_2.append(0)
        else:
            list_ABBBBA_1.append(0)
            list_ABBBBA_2.append(0)

        if (list_number[i][5] == list_number[i][8] and list_number[i][6] == list_number[i][7]):
            if (int(list_number[i][5]) < int(list_number[i][6])):
                list_ABBA_1.append(1)
                list_ABBA_2.append(0)
            elif(int(list_number[i][5]) > int(list_number[i][6])):
                list_ABBA_1.append(0)
                list_ABBA_2.append(1)
            else:
                list_ABBA_1.append(0)
                list_ABBA_2.append(0)
        else:
            list_ABBA_1.append(0)
            list_ABBA_2.append(0)
    df['list_ABBBBA_1']= list_ABBBBA_1
    df['list_ABBBBA_2']= list_ABBBBA_2
    df['list_ABBA_1']= list_ABBA_1
    df['list_ABBA_2']= list_ABBA_2
    return df

In [61]:

df = special_6_last(df)

In [62]:
df[df['list_ABBA_2'] == 1]

,Unnamed: 0,id,phone_number,price_vnd,type,length,pos_1,pos_2,pos_3,pos_4,...,kep_ong_dia,kep_ong_dia_last,dao_ong_dia_1,dao_ong_dia_1_last,dao_ong_dia_2,dao_ong_dia_2_last,list_ABBBBA_1,list_ABBBBA_2,list_ABBA_1,list_ABBA_2
416425,518673,sodepami_vn_0865148668,865148668,3730000,sodepami_v,9,8,6,5,1,...,0,0,0,0,0,0,0,0,0,1
358207,710092,sodepami_vn_0927867667,927867667,1100000,sodepami_v,9,9,2,7,8,...,0,0,0,0,0,0,0,0,0,1
734818,115133,sodepami_vn_0938183223,938183223,4000000,sodepami_v,9,9,3,8,1,...,0,0,0,0,0,0,0,0,0,1
193841,203510,sodepami_vn_0865794004,865794004,1340000,sodepami_v,9,8,6,5,7,...,0,0,0,0,0,0,0,0,0,1
145053,41130,sodepami_vn_0377005115,377005115,2050000,sodepami_v,9,3,7,7,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104058,726413,sodepami_vn_0929487667,929487667,1100000,sodepami_v,9,9,2,9,4,...,0,0,0,0,0,0,0,0,0,1
202694,801248,sodepami_vn_0375096006,375096006,1000000,sodepami_v,9,3,7,5,0,...,0,0,0,0,0,0,0,0,0,1
708446,565265,sodepami_vn_0325349559,325349559,500000,sodepami_v,9,3,2,5,3,...,0,0,0,0,0,0,0,0,0,1
372551,510624,sodepami_vn_0962176556,962176556,1650000,sodepami_v,9,9,6,2,1,...,0,0,0,0,0,0,0,0,0,1


In [63]:
def check_easy_remember(df):
    list_number = df['phone_number'].to_list()
    list_1_3_5_iden_bool = []
    list_1_3_5_iden_cat = []

    list_2_4_6_iden_bool = []
    list_2_4_6_iden_cat = []

    list_3_5_7_iden_bool = []
    list_3_5_7_iden_cat = []

    list_4_6_8_iden_bool = []
    list_4_6_8_iden_cat = []

    list_5_7_9_iden_bool = []
    list_5_7_9_iden_cat = []

    for i in range(len(list_number)):
        if ( (list_number[i][0] == list_number[i][2]) and (list_number[i][0] == list_number[i][4])):
            list_1_3_5_iden_bool.append(1)
            list_1_3_5_iden_cat.append((int)(list_number[i][0]))
        else:
            list_1_3_5_iden_bool.append(0)
            list_1_3_5_iden_cat.append(-1)

        if ( (list_number[i][1] == list_number[i][3]) and (list_number[i][1] == list_number[i][5])):
            list_2_4_6_iden_bool.append(1)
            list_2_4_6_iden_cat.append((int)(list_number[i][1]))
        else:
            list_2_4_6_iden_bool.append(0)
            list_2_4_6_iden_cat.append(-1)

        if ((list_number[i][2] == list_number[i][4]) and (list_number[i][2] == list_number[i][6])):
            list_3_5_7_iden_bool.append(1)
            list_3_5_7_iden_cat.append((int)(list_number[i][2]))
        else:
            list_3_5_7_iden_bool.append(0)
            list_3_5_7_iden_cat.append(-1)

        if ((list_number[i][3] == list_number[i][5]) and (list_number[i][3] == list_number[i][7])):
            list_4_6_8_iden_bool.append(1)
            list_4_6_8_iden_cat.append((int)(list_number[i][3]))
        else:
            list_4_6_8_iden_bool.append(0)
            list_4_6_8_iden_cat.append(-1)

        if ( (list_number[i][4] == list_number[i][6]) and (list_number[i][4] == list_number[i][8])):
            list_5_7_9_iden_bool.append(1)
            list_5_7_9_iden_cat.append((int)(list_number[i][4]))
        else:
            list_5_7_9_iden_bool.append(0)
            list_5_7_9_iden_cat.append(-1)


    df['1_3_5_iden_bool'] = list_1_3_5_iden_bool
    df['1_3_5_iden_cat'] = list_1_3_5_iden_cat

    df['2_4_6_iden_bool'] = list_2_4_6_iden_bool
    df['2_4_6_iden_cat'] = list_2_4_6_iden_cat

    df['3_5_7_iden_bool'] = list_3_5_7_iden_bool
    df['3_5_7_iden_cat'] = list_3_5_7_iden_cat

    df['4_6_8_iden_bool'] = list_4_6_8_iden_bool
    df['4_6_8_iden_cat'] = list_4_6_8_iden_cat

    df['5_7_9_iden_bool'] = list_5_7_9_iden_bool
    df['5_7_9_iden_cat'] = list_5_7_9_iden_cat
    
    return df

In [64]:
df = check_easy_remember(df)

In [65]:
LIST_FEAT1 = ['pos_' + str(i) for i in range(1,10)]
LIST_FEAT2 = ['phong_thuy_1','phong_thuy_2','4_last','5_last','3_last_iden','3_last_increasing',
              '4_last_iden', '5_last_iden' ,'4_last_increasing', 
              '5_last_increasing','max_count','list_same','palindrome',
              'nam_sinh','2_iden','2_lucky']
LIST_FEAT3 = ['count_' + str(i) for i in range(0,10)]
LIST_FEAT4 = ['palindrome_length','list_same_substr_length','length']
LIST_FEAT5 = ['pos_' + str(i) + "_" +str(j) for i in range(1,9) for j in range(i+1,10)]
LIST_FEAT6 = ['pos_'+str(i)+"_"+str(j)+"_"+str(k) for i in range(1,8) 
              for j in range(i+1,9) for k in range(j+1,10)]

LIST_FEAT9 = ['kep_duoi_cat','lap_dau_cat','lap_cuoi_cat','phat_loc_4_cat',
              'phat_loc_2_cat','5_combo_cat','6_combo_cat','3_combo_cat','4_combo_cat']

LIST_FEAT10 = ['list_ABBBBA_1','list_ABBBBA_2','list_ABBA_1','list_ABBA_2','lap_ong_dia','lap_ong_dia_last'
               ,'kep_ong_dia','kep_ong_dia_last','dao_ong_dia_1','dao_ong_dia_1_last','dao_ong_dia_2',
               'dao_ong_dia_2_last','2_lap_t1','2_lap_t2','2_lap_t3','2_lap_t4','kep_duoi_bool'
               ,'lap_dau_bool','lap_cuoi_bool','triple_2_last_r1','list_double_3_last','phat_loc_2_bool',
               'phat_loc_4_bool','nam_sinh_bool','bool_palindrome','bool_substr','2_lucky_bool','2_iden_bool','3_last_iden_bool',
                '4_last_iden_bool','5_last_iden_bool','3_last_increasing_bool','4_last_increasing_bool',
              '5_last_increasing_bool','1_3_5_iden_bool','2_4_6_iden_bool','3_5_7_iden_bool','4_6_8_iden_bool','5_7_9_iden_bool',
              '3_combo_bool','4_combo_bool','5_combo_bool','6_combo_bool','3_combo_s','3_combo_57','3_combo_689','4_combo_s',
               '4_combo_57','4_combo_689','list_double_3_increasing','list_double_4_increasing']

# LIST_FEAT7 = ['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l) for i in range(1,7) 
#               for j in range(i+1,8) for k in range(j+1,9) for l in range(k+1,10)]

# LIST_FEAT7 = ['pos_'+str(i)+"_"+str(i+1)+"_"+str(i+2)+"_"+str(i+3) for i in range(1,7)]

# LIST_FEAT8 = ['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l)+"_"+str(m) for i in range(1,6)
#               for j in range(i+1,7) for k in range(j+1,8) for l in range(k+1,9)
#               for m in range(l+1,10)]



class Categorical_encoder:
    def __init__(self):
        self.list_feat =LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT9
        self.list_encoder = []
    
    def check_containing_columns(self,list_columns):

        for i in self.list_feat:
            for j in range(len(list_columns)):
                if i == list_columns[j]:
                    break
                if j == len(list_columns) -1:
                    return False
        
        return True
                    
    def fit(self,df):
        
        if self.check_containing_columns(df.columns) is False:
               raise Exception("CAN NOT FIT BECAUSE DATAFRAME DO NOT HAVING ENOUGH REQUIRED COLUMNS")
        list_tmp = ['pos_' + str(i) for i in range(3,10)]
        list_tmp = list_tmp
        tmp = ""
        for i in self.list_feat:
            if (i in list_tmp):
                self.list_encoder.append(tmp)
                continue
            ordinal_encoder= preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value',
                                 unknown_value=-1)

            ordinal_encoder.fit(np.array(df[i]).reshape(-1,1))
            self.list_encoder.append(ordinal_encoder)
            tmp = ordinal_encoder

    
    def encode(self,df):
        list_tmp = ['pos_' + str(i) for i in range(3,10)]
        for i in range(len(self.list_feat)):
            #print(self.list_feat[i])
            df[self.list_feat[i]] = self.list_encoder[i].transform(np.array(df[self.list_feat[i]]).reshape(-1, 1)).flatten()
            #df[self.list_feat[i]] = self.list_encoder[i].transform(df[self.list_feat[i]])
        return df



In [66]:
list_features = LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT3 +  LIST_FEAT4 + LIST_FEAT5 + LIST_FEAT6 +LIST_FEAT9 + LIST_FEAT10

In [67]:
X_train, X_test,y_train, y_test = train_test_split(df[list_features],df['label'], test_size=0.1, random_state=25)

In [68]:
X_train.shape

(566085, 218)

In [69]:
# ct = Categorical_encoder()
# ct.fit(df[list_features])
# X_train = ct.encode(X_train)
# X_test = ct.encode(X_test)


In [70]:
del df
gc.collect()

118

In [71]:
# import xgboost as xgb

# reg = xgb.XGBClassifier(n_estimators = 400, max_depth=18,tree_method="gpu_hist")

# reg.fit(X_train, y_train)
# y_train_predict = reg.predict(X_train)
# print("XGB Accuracy Score of train set-> ",accuracy_score(y_train, y_train_predict)*100)

# y_test_predict = reg.predict(X_test)
# print(classification_report(y_test,y_test_predict))

In [72]:
# from sklearn.metrics import plot_confusion_matrix
# fig, ax = plt.subplots(figsize=(40, 30))
# plt.grid(False)
# sns.set(font_scale=1.5)
# plot_confusion_matrix(reg, X_test[list_features], y_test,cmap="OrRd",ax = ax)  
# plt.show()


In [73]:
# plt.figure(figsize = (50,60))
# plt.grid(False)

# sorted_idx = reg.feature_importances_.argsort()
# plt.barh(X_train.columns[sorted_idx], reg.feature_importances_[sorted_idx])
# plt.xlabel("XGB Feature Importance")

In [74]:
# import pickle
# pickle.dump(ct, open("encoder.pkl", "wb"))
# pickle.dump(reg, open("xgb_reg.pkl", "wb"))
# # pickle.dump(clf,open("ct.pkl", "wb"))

In [75]:
clf = CatBoostClassifier(task_type = "GPU", iterations = 1500,depth = 12,cat_features =LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT9)

# ,iterations=1000,learning_rate = 0.15,depth =10)    
# , ,LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT7 + LIST_FEAT8)
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
print("CBoost Accuracy Score of train set-> ",accuracy_score(y_train, y_train_predict)*100)

y_test_predict = clf.predict(X_test)
print(classification_report(y_test,y_test_predict))

Learning rate set to 0.168728
0:	learn: 1.5830561	total: 443ms	remaining: 11m 3s
1:	learn: 1.3571759	total: 876ms	remaining: 10m 56s
2:	learn: 1.2055810	total: 1.28s	remaining: 10m 40s
3:	learn: 1.0989496	total: 1.71s	remaining: 10m 40s
4:	learn: 1.0200378	total: 2.16s	remaining: 10m 46s
5:	learn: 0.9570278	total: 2.55s	remaining: 10m 34s
6:	learn: 0.9067830	total: 2.98s	remaining: 10m 36s
7:	learn: 0.8667918	total: 3.51s	remaining: 10m 54s
8:	learn: 0.8333051	total: 3.92s	remaining: 10m 48s
9:	learn: 0.8053045	total: 4.36s	remaining: 10m 50s
10:	learn: 0.7808026	total: 4.74s	remaining: 10m 42s
11:	learn: 0.7605673	total: 5.19s	remaining: 10m 43s
12:	learn: 0.7423029	total: 5.69s	remaining: 10m 51s
13:	learn: 0.7281207	total: 6.11s	remaining: 10m 48s
14:	learn: 0.7159473	total: 6.46s	remaining: 10m 39s
15:	learn: 0.7029280	total: 6.87s	remaining: 10m 37s
16:	learn: 0.6926875	total: 7.44s	remaining: 10m 49s
17:	learn: 0.6825208	total: 7.9s	remaining: 10m 50s
18:	learn: 0.6738237	total: 

In [76]:
import pickle
# pickle.dump(ct, open("encoder.pkl", "wb"))
# pickle.dump(reg, open("xgb_reg.pkl", "wb"))
pickle.dump(clf,open("cat.pkl", "wb"))

In [77]:
def guess_price(number, model):
    df = pd.DataFrame()
    df['phone_number'] = [number[1:]]
    df = removing_wrong_phone_number(df)
    df = creating_10_feats_from_single_pos(df)
    df = features_from_counting_numbers(df)
    df = sim_nam_sinh(df)
    df = symetric_4length_and_samesubtring_features(df)
    df = combo_feat(df)
    df = count_combo_features(df)
    df = features_2_inden_and_2_lucky_last(df)
    df = last_special_case(df)
    df = features_from_sum(df)
    df = creating_features_by_combine_pos(df)
    df = sim_loc_phat(df)
    df = double_3_last(df)
    df = triple_2_last(df)
    df = sim_6_last(df)
    df = special_4_sim(df)
    df = ong_dia(df)
    df = special_6_last(df)
    df = increasing_3_4(df)
    df = check_easy_remember(df)
    list_features = LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT3 +  LIST_FEAT4 + LIST_FEAT5 + LIST_FEAT6 +LIST_FEAT9 + LIST_FEAT10
    ans = model.predict(df[list_features])
 
    if(ans == 0):
        print("giá từ 0 đến 1,6tr vnd")
    elif ans == 1:
        print("giá từ 1,6 đến 3,6tr vnd")
    elif ans == 2:
        print("giá từ 3,6tr đến 6,5tr vnd")
    elif ans == 3:
        print("giá từ 6,5tr đến 10,8tr vnd")
    elif ans == 4:
        print("giá từ 10,8tr đến 15tr vnd")
    elif ans == 5:
        print("giá từ 15tr đến 61tr vnd")
    elif ans == 6:
        print("giá từ 61tr đến 150tr vnd")
    elif ans == 7:
        print("giá từ 150tr đến 310tr vnd")
    elif ans == 8:
        print("giá từ 310tr đến 530tr vnd")
    else:
        print("giá hơn 530tr")


In [78]:
import pickle
a = pickle.load(open("./cat.pkl", "rb"))
# encoder = pickle.load(open("./encoder.pkl", "rb"))

In [79]:
import time
start_time = time.time()
guess_price("0963126868",a)
print("run time is " + str(time.time() - start_time))

giá từ 15tr đến 61tr vnd
run time is 0.20430469512939453


In [80]:
# b = pickle.load(open("./cat.pkl", "rb"))
# encoder = pickle.load(open("./encoder.pkl", "rb"))